# Pre-process all the data

In [1]:
# Corpus reader:
import numpy as np
import random

import os
root = './Confs_newline/Conf1/'
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
reader = CategorizedPlaintextCorpusReader(root, r'.*\.txt', cat_pattern=r'(\w+)/*', encoding='utf-8')

In [2]:
print(reader.categories())
print(reader.fileids())

['kiz', 'kork', 'mutlu']
['kiz.txt', 'kork.txt', 'mutlu.txt']


In [3]:
### First, tokenize Punctuation: 
# create a token dictionary:
punc_dict= {'.':'||PERIOD||', ',': '||COMMA||', '"': '||QUOTATION_MARK||', ';': '||SEMICOLON||',
                '!': '||EXCLAMATION_MARK||', '?': '||QUESTION_MARK||', '(': '||LEFT_PAREN||',
                ')': '||RIGHT_PAREN||', '?': '||QUESTION_MARK||', 
                '\n': '||NEW_LINE||', '-': '||DASH||'}

In [4]:
def sent_tokenize_whole_tweets(text): # raw text --> whole tweets file content
    for key, token in punc_dict.items():
        text = text.replace(key, ' {} '.format(token))

    sentences= []
    for line in text.split('||NEW_LINE||'):
        line= line.strip()
        sentences.append(line)
    return sentences

In [5]:
all_text=[]
labels= []

In [6]:
for label,file_name in zip(reader.categories(), reader.fileids()):
    sentences= sent_tokenize_whole_tweets(reader.raw(file_name)) # --> this should return a list of contents
    labels.extend([label for i in sentences])
    all_text.extend([i.lower() for i in sentences])

# Now, we have all tweets in all_text list!

## Transforming Text into Numbers

In [7]:
word_counts={}
for i in range(len(all_text)):
    for word in all_text[i].split(" "):
        word_counts[word] = word_counts.get(word,0) +1

vocab = set(word_counts.keys())
vocab_size = len(vocab)

print("Number of unique words: {} ".format(vocab_size))
print("Number of tweets: {}".format(len(all_text)))
assert len(all_text)== len(labels), "Each tweet should have a label."

sorted_word_counts= sorted(word_counts, key= word_counts.get, reverse= True)

int_to_vocab= {ii: word for ii,word in enumerate(sorted_word_counts)}
vocab_to_int= {word: ii for ii, word in int_to_vocab.items()}

Number of unique words: 7426 
Number of tweets: 2170


In [8]:
all_text[2]

'hayır çok severim diziyi yani aşırı kızdım'

In [9]:
int_to_vocab[0]

''

# Reducing Noise in the Input Data

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

In [10]:
int_words = [vocab_to_int[word] for word in vocab]
print(int_words[:5])

[0, 4323, 5214, 2526, 6413]


In [11]:
threshold = 1e-4

word_counts_intwords = {vocab_to_int[word]:count for word,count in word_counts.items()}
total_count = vocab_size
freqs = {word: count/total_count for word, count in word_counts_intwords.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts_intwords}
# discard some frequent words, according to the subsampling equation
# create a new list of words for training

selected_words = [word for word in word_counts_intwords if random.random() < (1 - p_drop[word])]
print(selected_words[:30])

[2139, 291, 2141, 2142, 2143, 2144, 2145, 843, 632, 1226, 1, 633, 2147, 2148, 2150, 1228, 1229, 2151, 2152, 2154, 2155, 37, 2156, 2157, 63, 2158, 2159, 2160, 2161, 638]


In [12]:
len(selected_words)

5571

In [13]:
all_text[0]

'çok kızdım ne kadar ayıp şey'

In [14]:
for i in range(len(all_text)):
    line_split= all_text[i].split(" ")
    line_split_reduced= [word for word in line_split if vocab_to_int[word] in selected_words]
    all_text[i]= ' '.join(line_split_reduced)


# Delete empty tweets:

In [15]:
len(all_text)

2170

In [16]:
all_text= [line for line in all_text if len(line)>0]
labels= [labels[i] for i in range(len(all_text)) if len(all_text[i])>0]

print("Number of tweets after reducing some noise: {}".format(len(all_text)))
assert len(all_text)== len(labels), "Each tweet should have a label."

Number of tweets after reducing some noise: 2103


# Encoding the words
The embedding lookup requires that we pass in integers to our network. The easiest way to do this is to create dictionaries that map the words in the vocabulary to integers. Then we can convert each of our reviews into integers so they can be passed into the network.

In [17]:
word_counts={}
for i in range(len(all_text)):
    for word in all_text[i].split(" "):
        word_counts[word] = word_counts.get(word,0) +1

vocab = set(word_counts.keys())
vocab_size = len(vocab)
print("Number of unique words: {} ".format(vocab_size))

sorted_word_counts= sorted(word_counts, key= word_counts.get, reverse= True)

int_to_vocab= {ii: word for ii,word in enumerate(sorted_word_counts, 0)} #start from 1.
vocab_to_int= {word: ii for ii, word in int_to_vocab.items()}


Number of unique words: 5571 


In [18]:
## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
encoded_tweets = []
for tweet in all_text:
    encoded_tweets.append([vocab_to_int[word] for word in tweet.split(" ")])

# Encoding the labels

In [19]:
def map_label(l):
    if l== 'kiz':
        return 0
    elif l== 'kork':
        return 1
    return 2  

In [20]:
encoded_labels = np.array([map_label(label) for label in labels])

# Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some seq_length, we'll pad with 0s. For reviews longer than seq_length, we can truncate them to the first seq_length words. A good seq_length, in this case, is 200.

    Exercise: Define a function that returns an array features that contains the padded data, of a standard size, that we'll pass to the network.

        The data should come from review_ints, since we want to feed integers to the network.
        Each row should be seq_length elements long.
        For reviews shorter than seq_length words, left pad with 0s. That is, if the review is ['best', 'movie', 'ever'], [117, 18, 128] as integers, the row will look like [0, 0, 0, ..., 0, 117, 18, 128].
        For reviews longer than seq_length, use only the first seq_length words as the feature vector.

As a small example, if the seq_length=10 and an input review is:

[117, 18, 128]

The resultant, padded sequence should be:

[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]

Your final features array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified seq_length.

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [ ]:
max_lenght= max([len(i) for i in encoded_tweets])
max_lenght

In [ ]:
min_lenght= min([len(i) for i in encoded_tweets])
min_lenght

In [ ]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [ ]:
# Test your implementation!

seq_length = 10

features = pad_features(encoded_tweets, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(encoded_tweets), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

In [ ]:
features.shape

### Training, Validation, Test
split data into training, validation, and test sets.

In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

DataLoaders and Batching

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 20

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

# Word2Vec model:

In [21]:
import numpy as np
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [24]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [23]:
x,y = next(get_batches(encoded_tweets, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [[1039], [1039], [1039], [57, 1040, 1041, 1042, 1043, 1044], [57, 1040, 1041, 1042, 1043, 1044], [273, 182], [273, 182], [273, 182], [467, 0, 183, 1045, 1046, 1047], [467, 0, 183, 1045, 1046, 1047], [467, 0, 183, 1045, 1046, 1047]]
y
 [[57, 1040, 1041, 1042, 1043, 1044], [273, 182], [467, 0, 183, 1045, 1046, 1047], [1039], [273, 182], [1039], [57, 1040, 1041, 1042, 1043, 1044], [467, 0, 183, 1045, 1046, 1047], [1039], [57, 1040, 1041, 1042, 1043, 1044], [273, 182]]


In [25]:
extended_tweets= []
for line in encoded_tweets:
    extended_tweets.extend(line)

In [26]:
extended_tweets

[1039,
 57,
 1040,
 1041,
 1042,
 1043,
 1044,
 273,
 182,
 467,
 0,
 183,
 1045,
 1046,
 1047,
 0,
 468,
 469,
 468,
 1048,
 1049,
 1050,
 1051,
 5,
 0,
 0,
 1052,
 1053,
 9,
 1054,
 1055,
 1056,
 1057,
 184,
 1058,
 470,
 1059,
 1060,
 273,
 30,
 0,
 1061,
 471,
 185,
 472,
 6,
 6,
 473,
 1062,
 1063,
 1064,
 1065,
 1066,
 0,
 145,
 1067,
 474,
 0,
 1068,
 1069,
 274,
 1070,
 475,
 186,
 102,
 476,
 476,
 3,
 275,
 0,
 477,
 478,
 1071,
 1072,
 1073,
 1074,
 0,
 1075,
 276,
 1076,
 0,
 121,
 1077,
 0,
 122,
 146,
 1078,
 9,
 479,
 1079,
 1080,
 20,
 0,
 480,
 31,
 31,
 1081,
 31,
 1082,
 31,
 31,
 480,
 31,
 31,
 481,
 0,
 7,
 1083,
 1084,
 482,
 482,
 1085,
 3,
 483,
 0,
 0,
 1086,
 1087,
 0,
 0,
 1088,
 484,
 1089,
 0,
 0,
 0,
 1090,
 485,
 0,
 147,
 0,
 486,
 0,
 0,
 0,
 27,
 1091,
 0,
 0,
 183,
 37,
 123,
 1092,
 1093,
 103,
 104,
 1094,
 277,
 1095,
 1096,
 1097,
 187,
 44,
 1098,
 1099,
 1100,
 1101,
 0,
 1102,
 1103,
 0,
 188,
 45,
 1104,
 1105,
 1106,
 23,
 0,
 23,
 487,
 110

# The model:
- The input words are passed in as batches of input word tokens.
- This will go into a hidden layer of linear units (our embedding layer).
- Then, finally into a softmax output layer.

## SkipGram model

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

In [27]:
def cosine_similarity(embedding, valid_size=16, valid_window=10, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

In [28]:
import torch
from torch import nn
import torch.optim as optim

In [29]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

### Training
Because we applied a softmax function to our model output, we are using NLLLoss** as opposed to cross entropy. This is because Softmax  in combination with NLLLoss = CrossEntropy loss .

In [ ]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=100 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 30
steps = 10
epochs = 200

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(extended_tweets, 32):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(3) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

şu | ak, kemal
sonra | tanıştırdı, kansız
allaş | boksa, digor
çıktı | gönderme, üslupsan
hoşuma | olmamı, sevgidir
koptu | giydiriyorum, kanişimle
sinir | hata, muamelesi
||period|| | kollarım, whatsapp
şalgam | salakça, dön
sağal | koltuk, ardayı
analı | bebeğim, arkadaşlarımla
basıma | rabi'mden, kuzenimle
kulüpte | siniz, bahsede
büyütsün | yazılmaz, geçmek
sağlıkla | snsmksjs, şaapma
kapalım | pembe, i̇brahim
...
çıktı | gönderme, üslupsan
koptu | giydiriyorum, kanişimle
sinir | hata, muamelesi
allaş | digor, boksa
hoşuma | olmamı, sevgidir
sabah | sürede, siktirgit
sonra | tanıştırdı, kansız
en | kenarı, adet
kulüpte | siniz, bahsede
kapalım | pembe, i̇brahim
şalgam | salakça, dön
babalı | ritmi, hemcinsin
sağal | koltuk, ardayı
gerine | geceyi, şeyeydi
analı | bebeğim, arkadaşlarımla
sağlıkla | snsmksjs, şaapma
...
koptu | giydiriyorum, kanişimle
||period|| | kafeslerden, kollarım
sonra | tanıştırdı, kansız
sabah | sürede, siktirgit
sinir | hata, muamelesi
çıktı | gönderme, üslu

hoşuma | olmamı, açıldı
sinir | hata, ister
şu | ak, ndoye
sonra | kansız, tolga
||period|| | kafeslerden, malum
en | kenarı, bole
çıktı | gönderme, kendim
koptu | kanişimle, giydiriyorum
büyütsün | yazılmaz, ydi
sağal | sınırlarım, koltuk
babalı | ritmi, sarma
basıma | rabi'mden, kuzenimle
analı | bebeğim, duyarlı
şalgam | salakça, youtubea
sağlıkla | snsmksjs, hayal
gerine | geceyi, şeyeydi
...
sabah | vallahi, sürede
çıktı | gönderme, kendim
hoşuma | olmamı, açıldı
allaş | digor, boksa
||period|| | kafeslerden, malum
en | kenarı, bole
şu | ak, ndoye
sinir | hata, ister
büyütsün | yazılmaz, ydi
analı | bebeğim, duyarlı
sağal | koltuk, sınırlarım
şalgam | salakça, youtubea
basıma | rabi'mden, kuzenimle
kapalım | pembe, i̇brahim
gerine | geceyi, şeyeydi
babalı | ritmi, sarma
...
koptu | duramayan, kanişimle
allaş | digor, boksa
||period|| | kafeslerden, öpen
sinir | hata, ister
hoşuma | olmamı, açıldı
çıktı | gönderme, kendim
şu | ak, kemal
sonra | kansız, tolga
kulüpte | kansızlar, ba

||period|| | kafeslerden, malum
allaş | boksa, digor
sinir | ister, hata
koptu | duramayan, seveceksin
en | kenarı, bole
sabah | vallahi, siktirgit
çıktı | gönderme, kendim
hoşuma | olmamı, açıldı
analı | sağlıkla, bebeğim
babalı | sarma, içmeyeceksin
sağlıkla | snsmksjs, şaapma
büyütsün | yazılmaz, izlemeyin
basıma | rabi'mden, asmış
kulüpte | kansızlar, açtı
sağal | kuzen, taraftar
şalgam | salakça, yarabbim
...
hoşuma | olmamı, açıldı
çıktı | gönderme, kendim
en | kenarı, bole
sabah | vallahi, siktirgit
şu | ak, kemal
koptu | duramayan, seveceksin
||period|| | kafeslerden, malum
sinir | ister, hata
kulüpte | kansızlar, açtı
sağlıkla | snsmksjs, şaapma
analı | sağlıkla, bebeğim
büyütsün | yazılmaz, izlemeyin
babalı | sarma, içmeyeceksin
gerine | geceyi, şeyeydi
kapalım | pembe, şoförü
şalgam | salakça, yarabbim
...
hoşuma | olmamı, açıldı
çıktı | gönderme, kendim
allaş | boksa, digor
sabah | vallahi, antrenmanı
sonra | kansız, tolga
sinir | ister, hata
en | kenarı, bole
koptu | duram

en | kenarı, senenin
şu | ak, şeyim
sabah | vallahi, antrenmanı
sinir | ister, hata
koptu | duramayan, giydiriyorum
sonra | kansız, tolga
hoşuma | olmamı, paniğe
||period|| | kafeslerden, öpen
analı | sağlıkla, ertem
sağal | büzsüz, taraftar
şalgam | salakça, yarabbim
gerine | geceyi, şeyeydi
basıma | rabi'mden, asmış
sağlıkla | snsmksjs, analı
babalı | sarma, içmeyeceksin
kapalım | şoförü, i̇brahim
...
sabah | vallahi, antrenmanı
sinir | ister, hata
||period|| | kafeslerden, öpen
allaş | saatten, boksa
hoşuma | olmamı, paniğe
çıktı | gönderme, kendim
şu | ak, şeyim
en | kenarı, senenin
basıma | rabi'mden, asmış
kulüpte | kansızlar, vatandaş
babalı | sarma, içmeyeceksin
sağlıkla | snsmksjs, analı
kapalım | şoförü, i̇brahim
büyütsün | aynadan, yazılmaz
sağal | büzsüz, taraftar
gerine | geceyi, şeyeydi
...
allaş | saatten, boksa
koptu | duramayan, giydiriyorum
çıktı | gönderme, kendim
hoşuma | olmamı, paniğe
sinir | ister, hata
||period|| | kafeslerden, öpen
sabah | vallahi, antrenmanı
e

en | kenarı, senenin
sonra | kansız, komple
sinir | ister, hata
||period|| | kafeslerden, öpen
hoşuma | olmamı, paniğe
koptu | duramayan, giydiriyorum
şu | ak, sabret
allaş | saatten, ödevlerini
basıma | rabi'mden, darısı
sağal | kuzen, büzsüz
kapalım | şoförü, i̇brahim
büyütsün | aynadan, izlemeyin
babalı | sarma, cami
şalgam | salakça, yarabbim
sağlıkla | snsmksjs, analı
analı | sağlıkla, ertem
...
çıktı | gönderme, kendim
||period|| | kafeslerden, öpen
sinir | ister, hata
hoşuma | olmamı, paniğe
allaş | saatten, ödevlerini
sabah | vallahi, antrenmanı
en | kenarı, senenin
koptu | duramayan, giydiriyorum
analı | sağlıkla, ertem
büyütsün | aynadan, izlemeyin
sağlıkla | snsmksjs, analı
kulüpte | kansızlar, vatandaş
basıma | rabi'mden, darısı
babalı | sarma, cami
kapalım | şoförü, i̇brahim
sağal | kuzen, büzsüz
...
hoşuma | olmamı, paniğe
koptu | duramayan, giydiriyorum
çıktı | gönderme, kendim
sabah | vallahi, antrenmanı
şu | ak, sabret
sonra | kansız, komple
en | kenarı, senenin
allaş 

...

allaş | cehennemlikliyiz, çocuğumuz
şu | tüvitte, onunla
en | senenin, hmmm
sabah | demeye, kapatıyor
sonra | aldığı, mayına
çıktı | plaka, kürklü
sinir | ister, cık
hoşuma | açıkçası, atmazsam
kapalım | olmayınca, kaldın
sağlıkla | tatlısınız, büyütsün
şalgam | dkskdkfk, dili
gerine | yatağımda, bıraktım
sağal | aralıksız, sınavlarının
babalı | büyütsün, analı
basıma | hayvana, aydınlık
analı | büyütsün, diğerlerinin
...
||period|| | eonnie, öpen
sabah | demeye, kapatıyor
sonra | aldığı, mayına
allaş | cehennemlikliyiz, çocuğumuz
koptu | sevgilisine, kdkdkdkff
hoşuma | açıkçası, atmazsam
sinir | ister, cık
şu | tüvitte, onunla
büyütsün | analı, diğerlerinin
basıma | hayvana, aydınlık
şalgam | dkskdkfk, dili
analı | büyütsün, diğerlerinin
gerine | yatağımda, bıraktım
sağal | aralıksız, sınavlarının
sağlıkla | tatlısınız, büyütsün
kapalım | olmayınca, kaldın
...
allaş | çocuğumuz, cehennemlikliyiz
çıktı | plaka, kürklü
en | senenin, hmmm
sabah | demeye, kapatıyor
||period|| | eonnie, öp

||period|| | eonnie, öpen
sonra | aldığı, mayına
hoşuma | açıkçası, atmazsam
koptu | sevgilisine, kdkdkdkff
allaş | çocuğumuz, cehennemlikliyiz
sinir | ister, cık
en | senenin, hmmm
şu | tüvitte, edemedim
sağlıkla | tatlısınız, odaklı
gerine | yatağımda, bıraktım
sağal | aralıksız, sınavlarının
şalgam | dkskdkfk, dili
analı | büyütsün, diğerlerinin
basıma | eylesin, aydınlık
kulüpte | gerisini, demet
büyütsün | analı, babalı
...
sinir | ister, cık
koptu | sevgilisine, kdkdkdkff
şu | tüvitte, onunla
çıktı | plaka, kürklü
allaş | çocuğumuz, cehennemlikliyiz
sabah | demeye, kapatıyor
sonra | aldığı, mayına
en | senenin, hmmm
basıma | aydınlık, eylesin
analı | büyütsün, diğerlerinin
gerine | yatağımda, bıraktım
şalgam | dkskdkfk, dili
kapalım | olmayınca, kaldın
kulüpte | gerisini, demet
büyütsün | analı, babalı
babalı | büyütsün, analı
...
||period|| | öpen, eonnie
şu | tüvitte, onunla
hoşuma | açıkçası, atmazsam
koptu | sevgilisine, kdkdkdkff
sonra | aldığı, olmamı
en | senenin, hmmm
çık

koptu | sevgilisine, sürmektir
||period|| | öpen, eonnie
çıktı | plaka, kürklü
sabah | demeye, kapatıyor
şu | tüvitte, edemedim
sinir | ister, cık
hoşuma | açıkçası, atmazsam
en | senenin, hmmm
babalı | büyütsün, analı
sağlıkla | tatlısınız, odaklı
analı | büyütsün, diğerlerinin
kulüpte | gerisini, demet
kapalım | olmayınca, kaldın
gerine | yatağımda, halinize
şalgam | dkskdkfk, dili
basıma | eylesin, aydınlık
...
şu | tüvitte, edemedim
||period|| | öpen, eonnie
hoşuma | açıkçası, atmazsam
allaş | çocuğumuz, cehennemlikliyiz
sonra | aldığı, mayına
çıktı | plaka, kürklü
en | senenin, hmmm
sabah | demeye, kapatıyor
büyütsün | analı, babalı
sağlıkla | tatlısınız, odaklı
kapalım | olmayınca, kaldın
gerine | yatağımda, halinize
kulüpte | demet, gerisini
şalgam | dkskdkfk, dili
sağal | aralıksız, sınavlarının
analı | büyütsün, diğerlerinin
...
en | senenin, hmmm
sonra | aldığı, mayına
şu | tüvitte, edemedim
allaş | çocuğumuz, cehennemlikliyiz
sinir | ister, cık
koptu | sevgilisine, sürmektir

sonra | aldığı, olmamı
koptu | sürmektir, sevgilisine
sabah | demeye, kapatıyor
çıktı | plaka, kürklü
allaş | çocuğumuz, cehennemlikliyiz
en | senenin, hmmm
||period|| | eonnie, öpen
şu | tüvitte, onunla
basıma | aydınlık, eylesin
analı | büyütsün, diğerlerinin
babalı | büyütsün, analı
büyütsün | analı, diğerlerinin
gerine | yatağımda, halinize
kulüpte | demet, gerisini
sağlıkla | tatlısınız, odaklı
şalgam | dkskdkfk, açmışsın
...
hoşuma | açıkçası, atmazsam
sabah | demeye, kapatıyor
allaş | çocuğumuz, cehennemlikliyiz
sinir | ister, cık
sonra | aldığı, olmamı
çıktı | plaka, kürklü
||period|| | eonnie, öpen
şu | tüvitte, edemedim
sağlıkla | tatlısınız, odaklı
şalgam | dkskdkfk, dili
basıma | aydınlık, eylesin
gerine | yatağımda, halinize
kapalım | olmayınca, kaldın
kulüpte | demet, gerisini
sağal | aralıksız, sınavlarının
babalı | büyütsün, sağlıkla
...
koptu | sürmektir, sevgilisine
||period|| | öpen, eonnie
allaş | çocuğumuz, cehennemlikliyiz
çıktı | plaka, kürklü
sinir | ister, cık


sinir | ister, cık
şu | tüvitte, edemedim
hoşuma | açıkçası, atmazsam
çıktı | plaka, kürklü
sabah | demeye, kapatıyor
sonra | aldığı, yakalandım
koptu | sürmektir, sevgilisine
en | senenin, hmmm
kulüpte | demet, akalından
analı | büyütsün, diğerlerinin
babalı | büyütsün, analı
şalgam | dkskdkfk, dili
sağlıkla | tatlısınız, yaşayın
kapalım | olmayınca, kaldın
basıma | aydınlık, hayvana
sağal | aralıksız, sınavlarının
...
en | senenin, hmmm
çıktı | plaka, kürklü
koptu | sürmektir, sevgilisine
sonra | aldığı, yakalandım
hoşuma | açıkçası, atmazsam
||period|| | öpen, eonnie
sabah | demeye, kapatıyor
şu | tüvitte, edemedim
kulüpte | demet, akalından
büyütsün | analı, babalı
kapalım | olmayınca, kaldın
sağlıkla | tatlısınız, yaşayın
basıma | aydınlık, eylesin
gerine | yatağımda, halinize
sağal | aralıksız, sınavlarının
şalgam | dkskdkfk, dili
...
||period|| | öpen, eonnie
çıktı | plaka, kürklü
en | senenin, hmmm
sabah | demeye, kapatıyor
allaş | çocuğumuz, cehennemlikliyiz
hoşuma | açıkçası,